In [1]:
import os

In [2]:
%pwd

'c:\\Users\\MAINGATE\\DEsktop\\Maize_disease_classification\\end-to-end_mazie_disease_classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\MAINGATE\\DEsktop\\Maize_disease_classification\\end-to-end_mazie_disease_classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: tuple


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir : Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path




In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tf_keras as tf
import ast


[2025-05-05 23:51:08,686: WARNING: module_wrapper: From c:\Users\MAINGATE\.conda\envs\maize\lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):

        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config= self.config.prepare_callbacks

        model_ckpt_dir=os.path.dirname(config.checkpoint_model_filepath)

        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])


        prepare_callback_config= PrepareCallbacksConfig(
            root_dir= Path(config.root_dir),
            tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)

        )
        
        return prepare_callback_config



    
    def get_training_config(self) -> TrainingConfig:
        training= self.config.training
        prepare_base_model= self.config.prepare_base_model
        params= self.params
        training_data= os.path.join(self.config.data_ingestion.unzip_dir, "Maize_leaf_dataset")
        create_directories([
            Path(training.root_dir)
        ])


        training_config= TrainingConfig(
            root_dir = Path(training.root_dir),
            trained_model_path= Path(training.trained_model_path),
            base_model_path= Path(prepare_base_model.base_model_path),
            training_data= Path(training_data),
            params_epochs= params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size= ast.literal_eval(params.IMAGE_SIZE)
        )

        return training_config

In [8]:
import time

In [19]:
class PrepareCallback:
    def __init__(self, config:PrepareCallbacksConfig):
        self.config= config




    @property
    def _create_tb_callbacks(self):
        timestamp=time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir=os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.callbacks.TensorBoard(log_dir=tb_running_log_dir)


    @property
    def _create_ckpt_callbacks(self):
        return tf.callbacks.ModelCheckpoint(
            filepath= str(self.config.checkpoint_model_filepath),
            save_best_only= True
        )


    def get_tb_ckpt_callbacks(self):
        return [ 
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [20]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config=config

    
    def get_base_model(self):
        self.model=tf.models.load_model(
            self.config.base_model_path
        )

    
    def train_valid_generator(self):
        datagenerator_kwargs= dict(
            rescale= 1./255,
            validation_split=0.20
        )
        print("params_image_size =", self.config.params_image_size)


        dataflow_kwargs= dict(
            target_size= self.config.params_image_size[:2],
            batch_size= self.config.params_batch_size,
            interpolation= "bilinear"
        )


        valid_datagenerator= tf.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset= "validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator= tf.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip= True,
                width_shift_range= 0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range= 0.2,
                **datagenerator_kwargs
            )

        else:
            train_datagenerator= valid_datagenerator

        
        self.train_generator= train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset= "training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model:tf.Model):
        model.save(path)


    def train(self, callback_list:list):
        self.steps_per_epoch= self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps= self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs= self.config.params_epochs,
            steps_per_epoch= self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data= self.valid_generator,
            callbacks= callback_list
        )

        self.save_model(
            path= self.config.trained_model_path,
            model=self.model
        )

In [ ]:
try:
    config= ConfigurationManager()
    prepare_callbacks_config=config.get_prepare_callback_config()
    prepare_callbacks= PrepareCallback(config=prepare_callbacks_config)
    callback_list= prepare_callbacks.get_tb_ckpt_callbacks()

    training_config= config.get_training_config()
    training= Training(config= training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )

except Exception as e:
    raise e

[2025-05-06 00:00:36,147: INFO: common: yaml file: (path_to_yaml) loaded successfully]
[2025-05-06 00:00:36,188: INFO: common: yaml file: (path_to_yaml) loaded successfully]
[2025-05-06 00:00:36,202: INFO: common: created a directory at : artifacts]
[2025-05-06 00:00:36,213: INFO: common: created a directory at : artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-05-06 00:00:36,226: INFO: common: created a directory at : artifacts\training]


params_image_size = (224, 224, 3)
Found 1870 images belonging to 3 classes.
Found 7486 images belonging to 3 classes.
Epoch 1/16
[2025-05-06 00:06:19,424: WARNING: module_wrapper: From c:\Users\MAINGATE\.conda\envs\maize\lib\site-packages\tensorflow\python\autograph\converters\directives.py:126: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.
]
 71/233 [========>.....................] - ETA: 57:27 - loss: 0.2922 - accuracy: 0.8890

NameError: name 'self' is not defined